In [ ]:
import pandas as pd

# Create a sample DataFrame
data = {
    'layout': ['USA1', 'USA1', 'USA1', 'USA2', 'USA2', 'USA2', 'USA3', 'USA3', 'USA3'],
    'scenario': [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'historical': [True, True, False, True, False, False, True, False, False],
    'fast': [True, False, True, True, False, False, False, False, True],
    'big': [True, True, False, False, True, False, False, True, True],
}

df = pd.DataFrame(data)

import pandas as pd
import numpy as np
import random

# Define parameters
layouts = ['USA1', 'USA2', 'USA3']
scenarios = [1, 2, 3]
ground_sensors = ['strategy1', 'strategy2']
drones = ['dstrategy1', 'dstrategy2', 'dstrategy3']

# Generate all combinations
data = []
for layout in layouts:
    for scenario in scenarios:
        for gs in ground_sensors:
            for dr in drones:
                # Simulate detection time: 20% chance of being -1 (not detected)
                dt = round(random.uniform(0, 60), 1) if random.random() > 0.2 else -1
                data.append((layout, scenario, gs, dr, dt))

# Create DataFrame
results = pd.DataFrame(data, columns=['layout', 'scenario', 'ground_sensor', 'drone_strategy_strategy', 'detection_time'])

In [383]:
df = pd.read_csv('results/scenario_summary.csv')
results = pd.read_csv('results/combined_benchmark_results.csvRandomSensorPlacementStrategy_DroneRoutingMaxCoverageResetStaticGreedy')

In [384]:
df.layout_number.nunique() # 49 layouts, 7746 scenarios, 248 historical matches

49

In [385]:
# rename layout to layout_number and scenario to scenario_number
results.rename(columns={'layout': 'layout_number', 'scenario': 'scenario_number'}, inplace=True)

In [386]:
merged_df = df.merge(results, how='left', on=["layout_number","scenario_number"])

In [363]:
merged_df.head()

,layout_number,scenario_number,season_number,seasonal_match,historical_match,big_fire,small_fire,fast_fire,slow_fire,sensor_strategy,drone_strategy,delta_t,device,execution_time,fire_size_cells,fire_percentage,map_explored,total_distance
0,4,225,2.0,False,False,True,False,False,True,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,65.0,undetected,0.000002,43954.0,13.688997,0.009655,305211.0
1,4,640,2.0,False,False,True,False,False,True,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,11.0,ground sensor,0.000002,3139.0,0.977608,0.009655,52682.0
2,4,591,2.0,False,False,True,False,False,True,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,67.0,undetected,0.000003,43932.0,13.682145,0.009655,314540.0
3,4,993,2.0,False,False,True,False,False,True,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,45.0,undetected,0.000002,80172.0,24.968700,0.009655,211598.0
4,4,732,2.0,False,False,True,False,False,True,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,72.0,undetected,0.000002,33551.0,10.449095,0.009655,337910.0


In [391]:
merged_df.rename(columns={'delta_t': 'detection_time'}, inplace=True)
results.rename(columns={'delta_t': 'detection_time'}, inplace=True)

In [392]:
# if df.undetected is True, then detection_time should be -1

merged_df['detection_time'] = np.where(
    merged_df['device'] == 'undetected',
    -1,
    merged_df['detection_time']
)
results['detection_time'] = np.where(
    results['device'] == 'undetected',
    -1,
    results['detection_time']
)

In [393]:
# replace 72 in delta_time with -1
# merged_df['delta_t'] = merged_df['delta_t'].replace(72, -1)
# results['delta_t'] = results['delta_t'].replace(72, -1)

# Subgroup Score

In [415]:

# Make sure 'detection_time' is in your columns
assert 'detection_time' in merged_df.columns, "'detection_time' not found in DataFrame columns"

# Step 1: Filter out -1 values for mean and std calculations
filtered_df = merged_df[merged_df['detection_time'] != -1]

# Step 2: Group and aggregate only valid values
agg_df = filtered_df.groupby(['sensor_strategy', 'drone_strategy', 'fast_fire', 'big_fire']).agg(
    detection_time_mean=('detection_time', 'mean'),
    detection_time_std=('detection_time', 'std')
)

# Step 3: Format into a single string column
agg_df['detection_summary'] = agg_df.apply(
    lambda row: f"{row['detection_time_mean']:.2f} ({row['detection_time_std']:.2f})" 
    if pd.notnull(row['detection_time_mean']) and pd.notnull(row['detection_time_std']) 
    else "", axis=1
)

agg_df = agg_df.drop(columns=['detection_time_mean', 'detection_time_std'])


# Step 3: Define valid percentage calculation (based on original, unfiltered df)
def valid_percentage(subgroup):
    total = len(subgroup)
    valid = (subgroup != -1).sum()
    return (valid / total) * 100 if total > 0 else 0

# Apply percentage function on the unfiltered data
valid_pct = merged_df.groupby(['sensor_strategy', 'drone_strategy', 'fast_fire', 'big_fire'])['detection_time'].apply(valid_percentage)
valid_pct.name = 'detection_time_valid_pct'

# Step 4: Combine results
result = agg_df.join(valid_pct, how='outer').reset_index()

result['summary'] = result.apply(
    lambda row: f"{row['detection_time_valid_pct']:.0f}%\n{row['detection_summary']}" 
    if pd.notnull(row['detection_time_valid_pct']) and pd.notnull(row['detection_summary']) 
    else "", axis=1
)

result = result.drop(columns=['detection_summary', 'detection_time_valid_pct'])

In [398]:
merged_df.groupby(['sensor_strategy', 'drone_strategy', 'fast_fire', 'big_fire'])['detection_time'].mean()

sensor_strategy                drone_strategy                            fast_fire  big_fire
RandomSensorPlacementStrategy  DroneRoutingMaxCoverageResetStaticGreedy  False      False      -0.980583
                                                                                    True        9.514808
                                                                         True       False      -1.000000
                                                                                    True       -1.000000
Name: detection_time, dtype: float64

In [399]:
# Assuming 'result' is the DataFrame from previous steps:
pivoted_result = result.pivot_table(
    index=['sensor_strategy', 'drone_strategy'],
    columns=['fast_fire', 'big_fire'],
    values=['summary'],
    aggfunc='first'
)

# Optional: flatten MultiIndex columns for easier viewing
pivoted_result.columns = ['_'.join(map(str, col)) for col in pivoted_result.columns]
pivoted_result = pivoted_result.reset_index()

In [400]:
results_pt2 = pivoted_result

# Overall Score

In [401]:
results.rename(columns={'delta_t': 'detection_time'}, inplace=True)

In [416]:
valid_pct.head()

sensor_strategy                drone_strategy                            fast_fire  big_fire
RandomSensorPlacementStrategy  DroneRoutingMaxCoverageResetStaticGreedy  False      False        0.970874
                                                                                    True        28.418399
                                                                         True       False        0.000000
                                                                                    True         0.000000
Name: detection_time_valid_pct, dtype: float64

In [413]:
results_pt1 = results.groupby(['sensor_strategy', 'drone_strategy']).agg(
    detection_time_mean=('detection_time', 'mean'),
    detection_time_std=('detection_time', 'std')
)

results_pt1['detection_summary'] = results_pt1.apply(
    lambda row: f"{row['detection_time_mean']:.2f} ({row['detection_time_std']:.2f})" 
    if pd.notnull(row['detection_time_mean']) and pd.notnull(row['detection_time_std']) 
    else "", axis=1
)

results_pt1 = results_pt1.drop(columns=['detection_time_mean', 'detection_time_std'])

# Apply percentage function on the unfiltered data
valid_pct = results.groupby(['sensor_strategy', 'drone_strategy'])['detection_time'].apply(valid_percentage)
valid_pct.name = 'detection_time_valid_pct'

# Step 4: Combine results
result = results_pt1.merge(valid_pct, how='outer', on=['sensor_strategy', 'drone_strategy']).reset_index()

result['summary'] = result.apply(
    lambda row: f"{row['detection_time_valid_pct']:.0f}%\n{row['detection_summary']}" 
    if pd.notnull(row['detection_time_valid_pct']) and pd.notnull(row['detection_summary']) 
    else "", axis=1
)

result = result.drop(columns=['detection_summary', 'detection_time_valid_pct'])

result


,sensor_strategy,drone_strategy,summary
0,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,27%\n8.82 (19.57)


In [404]:
results_pt1 = result
results_pt1

,sensor_strategy,drone_strategy,summary
0,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,27%\n8.82 (19.57)


# Historical Score

In [405]:
merged_df.head()

,layout_number,scenario_number,season_number,seasonal_match,historical_match,big_fire,small_fire,fast_fire,slow_fire,sensor_strategy,drone_strategy,detection_time,device,execution_time,fire_size_cells,fire_percentage,map_explored,total_distance
0,4,225,2.0,False,False,True,False,False,True,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,-1.0,undetected,0.000002,43954.0,13.688997,0.009655,305211.0
1,4,640,2.0,False,False,True,False,False,True,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,11.0,ground sensor,0.000002,3139.0,0.977608,0.009655,52682.0
2,4,591,2.0,False,False,True,False,False,True,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,-1.0,undetected,0.000003,43932.0,13.682145,0.009655,314540.0
3,4,993,2.0,False,False,True,False,False,True,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,-1.0,undetected,0.000002,80172.0,24.968700,0.009655,211598.0
4,4,732,2.0,False,False,True,False,False,True,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,-1.0,undetected,0.000002,33551.0,10.449095,0.009655,337910.0


In [406]:
hist_df.detection_time.mean()

nan

In [407]:
hist_df = merged_df[merged_df['historical_match'] == True]

results_pt3 = hist_df.groupby(['sensor_strategy', 'drone_strategy']).agg(
    detection_time_mean=('detection_time', 'mean'),
    detection_time_std=('detection_time', 'std')
)

results_pt3['detection_summary'] = results_pt3.apply(
    lambda row: f"{row['detection_time_mean']:.2f} ({row['detection_time_std']:.2f})" 
    if pd.notnull(row['detection_time_mean']) and pd.notnull(row['detection_time_std']) 
    else "", axis=1
)
results_pt3 = results_pt3.drop(columns=['detection_time_mean', 'detection_time_std'])


# Apply percentage function on the unfiltered data
valid_pct = hist_df.groupby(['sensor_strategy', 'drone_strategy'])['detection_time'].apply(valid_percentage)
valid_pct.name = 'detection_time_valid_pct'

# Step 4: Combine results
result = results_pt3.join(valid_pct, how='outer').reset_index()

result['summary'] = result.apply(
    lambda row: f"{row['detection_time_valid_pct']:.0f}%\n{row['detection_summary']}" 
    if pd.notnull(row['detection_time_valid_pct']) and pd.notnull(row['detection_summary']) 
    else "", axis=1
)

result = result.drop(columns=['detection_summary', 'detection_time_valid_pct'])

results_pt3 = result
results_pt3

,sensor_strategy,drone_strategy,summary


In [408]:
results_merged = results_pt1.merge(results_pt2, how='left', on=['sensor_strategy', 'drone_strategy'])
    # results_pt3, how='left', on=['sensor_strategy', 'drone_strategy'], suffixes=('', '_historical')).merge(
        

In [409]:
results_merged

,sensor_strategy,drone_strategy,summary,summary_False_False,summary_False_True,summary_True_False,summary_True_True
0,RandomSensorPlacementStrategy,DroneRoutingMaxCoverageResetStaticGreedy,27%\n8.82 (19.57),1%\n,28%\n36.00 (20.92),,


In [410]:
results_merged.set_index(['sensor_strategy', 'drone_strategy'], inplace=True)

In [411]:
latex_str = results_merged.to_latex(
    index=True,
    escape=False,
    formatters={"name": str.upper},
    float_format="{:.3f}".format,
)

# Optional: escape percent signs
latex_str = latex_str.replace('%', r'\%')

# Insert \midrule after every data row (i.e., every '\\' that ends a row)
lines = latex_str.splitlines()
new_lines = []

for line in lines:
    new_lines.append(line)
    if line.strip().endswith('\\\\') and not line.strip().startswith(r'\toprule') and not line.strip().startswith(r'\midrule') and not line.strip().startswith(r'\bottomrule'):
        new_lines.append(r'\midrule')

# Join back into one LaTeX string
final_latex = '\n'.join(new_lines)

with open("leaderboards.txt", "w") as f:
    f.write(final_latex)